## Import Packages

In [1]:
using Pkg
Pkg.activate(".")
using Oceananigans #use v.1.10
using Oceananigans.Units
using Oceananigans.OutputReaders: FieldTimeSeries
using Oceananigans.BoundaryConditions
using Oceananigans.TurbulenceClosures
using Oceananigans.Architectures: GPU
#using CUDA: CuArray
using CairoMakie 
using NCDatasets

  Activating new project at `c:\Users\Tom Cummings\Documents\Oceananigans_projects\B00_runs`
┌ Warning: You are using Julia v1.11 or later!"
│ Oceananigans is currently tested on Julia v1.10."
│ If you find issues with Julia v1.11 or later,"
│ please report at https://github.com/CliMA/Oceananigans.jl/issues/new
└ @ Oceananigans C:\Users\Tom Cummings\.julia\packages\Oceananigans\3ZIHr\src\Oceananigans.jl:124


## Define Model Parameters

In [2]:
# Model parameters
const Nx = 500
const Nz = 160
const f = 1e-4               # Coriolis frequency [s⁻¹]
const L_front = 100kilometers  # Initial front width [m]
const aspect_ratio = 1000      # L/H
const δ = 0                   # Strain ratio (α/f)
const Ro = 1/2               # Rossby number (defines M^2)
const F = Inf              # Froude number (N² = M²/(F²H))
const Re_h = 1e10     # horizontal reynolds number
const Re_v = +Inf              #vertical reynolds number
const n = 2              #diffusivity number     

const sponge_width = 8kilometers
const damping_rate = f

# Derived parameters
H_front = L_front/aspect_ratio
α = f*δ
M² = (Ro^2*f^2*L_front)/H_front
N² = (M²*L_front)/(F^2*H_front)
Bu = Ro/F
Δb = M²*L_front
κh = (sqrt(Δb*H_front)*L_front^(n-1))/Re_h
κv = κh*(Re_h/Re_v)*(H_front/L_front)^n




filename = "δ="*string(δ)*"_Ro="*string(Ro)*"_F="*string(F)*"_Re_h="*string(Re_h)
filename = "frontogenesis_scaled_aspect_ratio"

println("Filename: ", filename)
println("\nDerived parameters:")
println("H = ", H_front/1000, " km")
println("α = ", α, " s⁻¹")
println("M² = ", M², " s⁻²")
println("N² = ", N², " s⁻²")
println("Bu = ", Bu)
println("κh = ", κh)
println("κv = ", κv)

Filename: frontogenesis_scaled_aspect_ratio

Derived parameters:
H = 0.1 km
α = 0.0 s⁻¹
M² = 2.5e-6 s⁻²
N² = 0.0 s⁻²
Bu = 0.0
κh = 5.0e-5
κv = 0.0


## Define Model Domain

In [3]:

Lz = H_front

grid = RectilinearGrid(size = (Nx, Nz), #in ST15 they use Nx = 200*L_front, Nz = 100*H_front
                       x = (-L_front/2, L_front/2),
                       z = (-Lz, 0),
                       topology = (Periodic, Flat, Bounded))

500×1×160 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CPU with 3×0×3 halo
├── Periodic x ∈ [-50000.0, 50000.0) regularly spaced with Δx=200.0
├── Flat y                           
└── Bounded  z ∈ [-100.0, 0.0]       regularly spaced with Δz=0.625

## Forcing Terms

In [4]:
# advective forcing term 
u_background = XFaceField(grid)
u_background .= - α * xnodes(grid, Face(), Center(), Center())
background_flow = AdvectiveForcing(u = u_background)

# no addtional u forcing

# v forcing
v_forcing_func(x, z, t, v, α) = - 2*α*v
v_forcing = Forcing(v_forcing_func, parameters = α, field_dependencies = :v )

# w forcing
w_forcing_func(x, z, t, w, α) = - α*w
w_forcing = Forcing(w_forcing_func, parameters = α, field_dependencies = :w )

# b forcing
b_forcing_func(x, z, t, α, b ) = - α*b
b_forcing = Forcing(b_forcing_func, parameters = α, field_dependencies= :b )



ContinuousForcing{Float64}
├── func: b_forcing_func (generic function with 1 method)
├── parameters: 0.0
└── field dependencies: (:b,)

## Sponge Layer

In [5]:
Δb = L_front * M²

target_buoyancy_left(x,z,t) = z*N²
target_buoyancy_right(x,z,t) = z*N² + Δb
    




left_mask_3D   = GaussianMask{:x}(center=-grid.Lx/2, width = sponge_width)
left_mask(x,z) = left_mask_3D(x,0,z)
uvw_sponge_left = Relaxation(rate=damping_rate, mask=left_mask)
b_sponge_left = Relaxation(rate=damping_rate, mask=left_mask, target = target_buoyancy_left) 

right_mask_3D  = GaussianMask{:x}(center=grid.Lx/2,width = sponge_width)
right_mask(x,z) = right_mask_3D(x,0,z)
uvw_sponge_right = Relaxation(rate=damping_rate, mask=right_mask)
b_sponge_right = Relaxation(rate=damping_rate, mask=right_mask, target = target_buoyancy_right ) 



Relaxation{Float64, typeof(right_mask), typeof(target_buoyancy_right)}
├── rate: 0.0001
├── mask: right_mask (generic function with 1 method)
└── target: target_buoyancy_right (generic function with 1 method)

## Boundary Conditions

In [6]:

using Oceananigans.BoundaryConditions

# Free-slip for u and v (∂u/∂z = ∂v/∂z = 0)
free_slip = FieldBoundaryConditions(
    top = GradientBoundaryCondition(0.0),
    bottom = GradientBoundaryCondition(0.0)
)

# No vertical flow (w = 0 at top/bottom)
no_penetration = FieldBoundaryConditions(
    top = ValueBoundaryCondition(0.0),
    bottom = ValueBoundaryCondition(0.0)
)

velocity_bcs = (
    u = free_slip,
    v = free_slip,
    w = no_penetration
)

(u = Oceananigans.FieldBoundaryConditions, with boundary conditions
├── west: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── east: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── south: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── north: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── bottom: GradientBoundaryCondition: 0.0
├── top: GradientBoundaryCondition: 0.0
└── immersed: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing), v = Oceananigans.FieldBoundaryConditions, with boundary conditions
├── west: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── east: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── south: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── north: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── bottom: GradientBoundaryCondition: 0.0
├── top: GradientBoundaryCondition: 0.0
└── immersed: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing), w = O

## Buoyancy BC's

In [7]:

∂b∂z_top = N²
∂b∂z_bottom = N²

buoyancy_bcs = FieldBoundaryConditions(
    top = GradientBoundaryCondition(∂b∂z_top),
    bottom = GradientBoundaryCondition(∂b∂z_bottom)
)
    

Oceananigans.FieldBoundaryConditions, with boundary conditions
├── west: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── east: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── south: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── north: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── bottom: GradientBoundaryCondition: 0.0
├── top: GradientBoundaryCondition: 0.0
└── immersed: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)

## Closure

In [8]:
horizontal_closure = HorizontalScalarDiffusivity(ν=κh, κ=κh )
vertical_closure = VerticalScalarDiffusivity(ν=κv , κ=κv )
closure = (horizontal_closure, vertical_closure)

(HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=5.0e-5, κ=5.0e-5), VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0, κ=0.0))

## Defining the Model

In [9]:
#with sponge

#=
model = NonhydrostaticModel(; grid,
                coriolis = FPlane(f = f),
                buoyancy = BuoyancyTracer(),
                tracers = :b,
                advection = WENO(),
                forcing = (; u = (background_flow, uvw_sponge_left, uvw_sponge_right),
                             v = (background_flow , v_forcing, uvw_sponge_left, uvw_sponge_right) , 
                             w = (background_flow , w_forcing, uvw_sponge_left, uvw_sponge_right),
                             b = (background_flow, b_forcing, b_sponge_left, b_sponge_right)),
                boundary_conditions = (; b=buoyancy_bcs, velocity_bcs),
                closure = closure

                )
=#

#without sponge 

#=
model = NonhydrostaticModel(; grid,
                coriolis = FPlane(f = f),
                buoyancy = BuoyancyTracer(),
                tracers = :b,
                advection = WENO()
                forcing = (; u = (background_flow), v = (background_flow , v_forcing) , w = (background_flow , w_forcing), b = (background_flow, b_forcing))
                #boundary_conditions = (; b=buoyancy_bcs, velocity_bcs)
                )
=#

#without forcing
model =  NonhydrostaticModel(; grid,
                coriolis = FPlane(f = f),
                buoyancy = BuoyancyTracer(),
                tracers = :b,
                advection = WENO(),
                #forcing = (; u = (background_flow), v = (background_flow , v_forcing) , w = (background_flow , w_forcing), b = (background_flow, b_forcing))
                #boundary_conditions = (; b=buoyancy_bcs, velocity_bcs)
)




NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 500×1×160 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CPU with 3×0×3 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: WENO{3, Float64, Float32}(order=5)
├── tracers: b
├── closure: Nothing
├── buoyancy: BuoyancyTracer with ĝ = NegativeZDirection()
└── coriolis: FPlane{Float64}(f=0.0001)

## Initial Conditions

In [10]:
#inital setup

Δb = L_front * M²  # buoyancy jump across front  

bᵢ(x, z) = Δb*((sin(2*pi/L_front * x) + 1)/2)

set!(model, b= bᵢ, u = 0, v = 0, w = 0)  # Start from rest


## Define Simulation

In [11]:
simulation = Simulation(model, Δt=20minutes, stop_time=10days)

Simulation of NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 20 minutes
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 10 days
├── Stop iteration: Inf
├── Wall time limit: Inf
├── Minimum relative step: 0.0
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => 4
│   ├── stop_iteration_exceeded => -
│   ├── wall_time_limit_exceeded => e
│   └── nan_checker => }
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

In [12]:
conjure_time_step_wizard!(simulation, IterationInterval(20), cfl=0.2, max_Δt=20minutes)

In [13]:
using Printf

wall_clock = Ref(time_ns())

function print_progress(sim)
    u, v, w = model.velocities
    progress = 100 * (time(sim) / sim.stop_time)
    elapsed = (time_ns() - wall_clock[]) / 1e9

    @printf("[%05.2f%%] i: %d, t: %s, wall time: %s, max(u): (%6.3e, %6.3e, %6.3e) m/s, next Δt: %s\n",
            progress, iteration(sim), prettytime(sim), prettytime(elapsed),
            maximum(abs, u), maximum(abs, v), maximum(abs, w), prettytime(sim.Δt))

    wall_clock[] = time_ns()

    return nothing
end

add_callback!(simulation, print_progress, IterationInterval(100))

## Diagnostic/Output

In [14]:
# Output setup
g = 9.81
u, v, w = model.velocities
ζ = ∂z(u) - ∂x(w)  # Vorticity in x-z plane
b = model.tracers.b

# Compute Ri
Ri = N² / (∂z(u)^2 + ∂z(v)^2) 
M_squared = ∂x(b)

#X = x + v/f
dbdX = ∂x(b)*(1/(1 + (1/f) * ∂x(v)))




#=
#For Julia animation
simulation.output_writers[:fields] = JLD2Writer(
    model, (; b, ζ , u, v, w),
    filename=filename * ".jld2",
    schedule=TimeInterval(0.5day),
    overwrite_existing=true
    )
=#
#=

#For Lagrangian filtering
simulation.output_writers[:fields] = JLD2Writer(
    model, (; b, u, v, w, Ri, M_squared, dbdX), filename=filename * ".jld2", schedule=TimeInterval(10minutes), overwrite_existing=true)
=#


#for python visualisation
simulation.output_writers[:fields] = NetCDFWriter(
    model, (; b, u, v, w, Ri, M_squared, dbdX), filename=filename * ".nc", schedule=TimeInterval(10minutes), overwrite_existing=true)

┌ Warning: Overwriting existing c:\Users\Tom Cummings\Documents\Oceananigans_projects\B00_runs\frontogenesis_scaled_aspect_ratio.nc.
└ @ OceananigansNCDatasetsExt C:\Users\Tom Cummings\.julia\packages\Oceananigans\3ZIHr\ext\OceananigansNCDatasetsExt.jl:1027


NetCDFWriter scheduled on TimeInterval(10 minutes):
├── filepath: frontogenesis_scaled_aspect_ratio.nc
├── dimensions: time(0), x_faa(500), x_caa(500), z_aaf(161), z_aac(160)
├── 7 outputs: (v, w, dbdX, Ri, b, u, M_squared)
└── array type: Array{Float32}
├── file_splitting: NoFileSplitting
└── file size: 34.4 KiB

## Run Simulation

In [15]:
@info "Running the simulation..."

run!(simulation)

@info "Simulation completed in " * prettytime(simulation.run_wall_time)

[ Info: Running the simulation...
[ Info: Initializing simulation...


[00.00%] i: 0, t: 0 seconds, wall time: 1.343 minutes, max(u): (0.000e+00, 0.000e+00, 0.000e+00) m/s, next Δt: 20 minutes


[ Info:     ... simulation initialization complete (31.058 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (14.004 seconds).


[00NaN%] i: 100, t: NaN days, wall time: 1.087 minutes, max(u): (   NaN,    NaN,    NaN) m/s, next Δt: NaN days


[ Info: time = NaN, iteration = 100: NaN found in field u. Stopping simulation.
[ Info: Simulation completed in 1.455 minutes


In [16]:
println( Δb)

0.25
